In [28]:
from pandas import read_csv
df = read_csv('dataset.csv', low_memory=False, index_col=0)

In [29]:
df.columns

Index(['created_at_first', 'category', 'is_business', 'price',
       'price[currency]', 'm', 'rooms_num', 'market', 'building_type',
       'floor_no',
       ...
       'access_types_soft_surfaced', 'deposit', 'deposit[currency]',
       'rent_to_students', 'city_lon', 'city_lat', 'city_name', 'district_lon',
       'district_lat', 'district_name'],
      dtype='object', length=110)

## Usunięcie zbędnych kolumn

In [30]:
df.dropna(thresh=10, axis=1, inplace=True)
df.drop(columns=[
    
    'price[currency]', 'rent[currency]', # nieistotne, (prawie) wszystko to PLN 
    'building_ownership', #????
    'building_floors_num',
    
    'garret_type', #??
    'is_bungalow', # ??
    'roofing', 'location','roof_type', 'recreational', # za dużo NA

    'access_types_asphalt', 
    'access_types_hard_surfaced', 
    'access_types_dirt' # przy tylu NA to pomijalne
    
], inplace=True)

## Powierzchnia

In [31]:
df.rename(columns={
    'm': 'area_inside', 'terrain_area': 'area_outside'
}, inplace=True)

## Status

In [32]:
df['construction_status'].replace('ready_to_use', 'ready')
df['construction_status'].replace('to_renovation', 'renovation')
df['construction_status'].replace('to_completion', 'unfinished')
df['construction_status'].replace('unfinished_open', 'unfinished')
df['construction_status'].replace('unfinished_close', 'unfinished')
df.rename(columns={'construction_status': 'status'}, inplace=True)

## Kategoria ogłoszenia

In [33]:
df['category'] = df['category'].replace('Mieszkanie na sprzedaż', 'flat')
df['category'] = df['category'].replace('Dom na sprzedaż', 'house')

## Ustalenie okresu ogłoszenia jako zmienne binarne

In [34]:
from pandas import to_datetime

df.rename(columns = { 'build_year': 'year' }, inplace=True)

df['created_at_first'] = to_datetime(df['created_at_first'])
df['upload_year'] = df['created_at_first'].dt.year.astype(float)
df['upload_month'] = df['created_at_first'].dt.month.astype(float)
df.drop('created_at_first', axis=1,inplace=True)

df['free_from'] = to_datetime(df['free_from'])
df['free_year'] = df['free_from'].dt.year.astype(float)
df['free_month'] = df['free_from'].dt.month.astype(float)
df.drop('free_from', axis=1, inplace=True)

## Liczba pięter i pokoi

In [35]:
df.rename(columns={ 'rooms_num': 'rooms' }, inplace=True)
df['rooms_num_more'] = (df['rooms'] == 'more')
df['rooms'] = df['rooms'].replace('more', 10)
df['rooms'] = df['rooms'].astype(float)

## Piętro

In [36]:
from numpy import nan

df.rename(columns={ 'floor_no': 'floor' }, inplace=True)

df['floor'] = df['floor'].replace('floor_', '', regex=True)
df['floor'] = df['floor'].replace('ground_floor', 0)
df['floor'] = df['floor'].replace('cellar', -1)
df['floor'] = df['floor'].replace('garret', nan)
df['floor'] = df['floor'].replace('higher_10', 10)
df['floor'] = df['floor'].astype(float)

## Piętra

In [37]:
from pandas import get_dummies

df['floors_num'] = df['floors_num'].replace('ground_floor', 'single')
df['floors_num'] = df['floors_num'].replace('one_floor', 'single')
df['floors_num'] = df['floors_num'].replace('two_floors', 'double')
df['floors_num'] = df['floors_num'].replace('more', 'multiple')

df = get_dummies(df, columns=['floors_num'], prefix='floor')

## Ogrzewanie

In [38]:
df['heating_gas'] =  ((df['heating'] == 'gas') | df['heating_types_gas']).astype(int)
df.drop(columns=['heating_types_gas'], inplace=True)

df['heating_urban'] =  ((df['heating'] == 'urban') | df['heating_types_urban']).astype(int)
df.drop(columns=['heating_types_urban'], inplace=True)

other_heating_values = ['other', 'electric', 'boiler_room', 'tiled_stove']
other_heating_cols = [
    'heating_types_fireplace', 
    'heating_types_electric',
    'heating_types_coal', 
    'heating_types_oil', 
    'heating_types_heat_pump',
    'heating_types_stove',
    'heating_types_solar_collector',
    'heating_types_biomass',
    'heating_types_geothermal'
]

df['heating_other'] =  (df['heating'].isin(other_heating_values) | df[other_heating_cols].sum(axis = 1)).astype(int)
df.drop(columns=['heating', *other_heating_cols], inplace=True)

## Płot

In [39]:
df['fence'] = df[df.filter(like='fence').columns].any(axis = 1).astype(int)
df.drop(columns=df.drop('fence', axis=1).filter(like='fence'), inplace=True)

## Dodatki

In [40]:
df['utility_furniture'] = df['equipment_types_furniture'].fillna(False).astype(int)
df['utility_equipment'] = df[[
    'equipment_types_stove',
    'equipment_types_oven',
    'equipment_types_fridge',
    'equipment_types_washing_machine',
    'equipment_types_dishwasher',
    'equipment_types_tv',
]].sum(axis=1).astype(int)
df.drop(columns=df.drop('utility_equipment', axis=1).filter(like='equipment'), inplace=True)

/tmp/ipykernel_8361/590735041.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['utility_furniture'] = df['equipment_types_furniture'].fillna(False).astype(int)


In [41]:
df['utility_connectivity'] = df[[
  'media_types_internet', 'media_types_cable-television', 'media_types_cable_television', 'media_types_phone'
]].sum(axis=1).astype(int)
df['utility_media'] = df[[
  'media_types_water', 'media_types_electricity', 'media_types_gas', 'media_types_sewage', 'media_types_cesspool', 'media_types_water_purification'
]].sum(axis=1).astype(int)

df.drop(columns=df.drop('utility_media', axis=1).filter(like='media'), inplace=True)

In [42]:
df['utility_entryphone'] = df['security_types_entryphone'].fillna(False).astype(int)
df['utility_security'] = df[['security_types_anti_burglary_door', 'security_types_monitoring', 'security_types_alarm']].sum(axis=1).astype(int)
df['utility_closed'] = df['security_types_closed_area'].fillna(False).astype(int)
df['utility_rollers'] = df['security_types_roller_shutters'].fillna(False).astype(int)

df.drop(columns=df.drop('utility_security', axis=1).filter(like='security'), inplace=True)

/tmp/ipykernel_8361/1358566043.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['utility_entryphone'] = df['security_types_entryphone'].fillna(False).astype(int)
/tmp/ipykernel_8361/1358566043.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['utility_closed'] = df['security_types_closed_area'].fillna(False).astype(int)
/tmp/ipykernel_8361/1358566043.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the f

In [43]:
df['utility_balcony'] = df[['extras_types_balcony', 'extras_types_terrace']].sum(axis=1).astype(int)
df['utility_garden'] = df[['extras_types_garden']].sum(axis=1).astype(int)
df['utility_pool'] = df[['extras_types_pool']].sum(axis=1).astype(int)
df['utility_lift'] = df[['extras_types_lift']].sum(axis=1).astype(int)
df['utility_space'] = df[['extras_types_basement','extras_types_attic', 'extras_types_usable_room']].sum(axis=1).astype(int)

df.drop(columns=df.filter(like='extras').columns, inplace=True)

## Otoczenie

In [44]:
df['vicinity_water'] = df[['vicinity_types_sea', 'vicinity_types_lake', 'vicinity_types_mountains']].sum(axis=1).astype(int)
# rzeki zaczynają się też w górach dlatego woda...
df['vicinity_forest'] = df['vicinity_types_forest'].fillna(False).astype(int)

df.drop(columns=df.filter(like='vicinity').columns, inplace=True)

/tmp/ipykernel_8361/734496406.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['vicinity_forest'] = df['vicinity_types_forest'].fillna(False).astype(int)


## Typ budynku

In [45]:
df['building_type'] = df['building_type'].replace([
    'apartment', 'detached', 'farm', 'house',
    'infill', 'loft', 'residence', 'ribbon',
    'semi_detached', 'separate', 'tenement', 'tenement_house',
], 'house')
df.rename(columns={'building_type': 'type'}, inplace=True)

## Okna

In [46]:
df['windows_type'] = df['windows_type'].replace([
    'aluminium', 'notany', 'wooden'
], 'other')
df.rename(columns={'windows_type': 'windows'}, inplace=True)

## Materiał

In [47]:
df['building_material'] = df['building_material'].replace([
    'breezeblock', 'hydroton', 'silikat', 'wood'
], 'other')

df['building_material'] = df['building_material'].replace([
    'cellular_concrete', 'concrete_plate', 'reinforced_concrete'
], 'concrete')

df.rename(columns={'building_material': 'material'}, inplace=True)

## Podsumowanie

In [48]:
numeric_cols = df.select_dtypes(include=['float', 'int']).columns
df = df[list(numeric_cols) + [col for col in df.sort_index(axis=1).columns if col not in ['price'] + list(numeric_cols)]]

In [49]:
ordered = ['price', 'rent', 'area_inside', 'area_outside', 'year', 'free_month', 'free_year', 'upload_month','upload_year']
ordered = [*ordered, *[a for a in df.sort_index(axis=1).columns if (a not in ordered)]]
df = df[ordered]

In [58]:
df_log = df.copy()

from numpy import log
df_log['price'] = df_log['price'].apply(lambda x: x if x == 0 else log(x))
df_log['rent'] = df_log['rent'].apply(lambda x: x if x == 0 else log(x))
df_log['area_inside'] = df_log['area_inside'].apply(lambda x: x if x == 0 else log(x))
df_log['area_outside'] = df_log['area_outside'].apply(lambda x: x if x == 0 else log(x))
df_log.rename(columns={ 'price': 'log_price', 'rent': 'log_rent', 'area_inside': 'log_area_inside', 'area_outside': 'log_area_outside' }, inplace=True)

In [61]:
from ydata_profiling import ProfileReport
report = ProfileReport(df_log, minimal=True)
report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df.to_csv('dataset.reduced.csv', index=False)